In [1]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup

In [2]:
naver_client_id = "kH1W4xEGrMFc13bhdu6E"
naver_client_secret = "RBv6Fm_uVJ"

# 블로그 크롤러 함수 정의

In [4]:
def naver_blog_crawling(search_blog_keyword, display_count, sort_type):
    search_result_blog_page_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)
    get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type)

# 크롤링 데이터 분석 함수

In [5]:
def get_blog_search_result_pagination_count(search_blog_keyword, display_count):
    encode_search_keyword = urllib.parse.quote(search_blog_keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_keyword
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))

        if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))

            if blog_pagination_total_count >= 1000:
                blog_pagination_count = 1000
            else:
                blog_pagination_count = blog_pagination_total_count

            print("키워드 " + search_blog_keyword + "에 해당하는 포스팅 수 : " + str(response_body_dict['total']))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 실제 페이징 수 : " + str(blog_pagination_total_count))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 처리할 수 있는 페이징 수 : " + str(blog_pagination_count))

        return blog_pagination_count

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\gudrb\AppData\Local\Temp\ipykernel_2572\2130282468.py:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if response_code is 200:


# 블로그 데이터 분석 함수

In [ ]:
# 네이버 블로그 데이터 같은 경우에는 url이 where 에는 post형태로 들어오고,
# 검색어는 query 뒤에 오는 형식이기 때문에 딕셔너리 형태로 parameter를 받는다.

# 네이버 검색 페이지에서 접근하는 방식

In [27]:
import requests

# url = 'https://section.blog.naver.com/'
url = 'https://search.naver.com/search.naver'
hrd ={'User-Agent': 'Mozilla/5.0', 'referer': 'http://naver.com'}
# 해당 hrd를 requests.get 모듈의 headers 인자로 넘기면 
# 서버에게 외부 프로그램이 아닌 정상적인 형태로 접근하는 것처럼 보일 수 있다고 한다.
param ={
    'where' : 'blog',
    'query' : '체크인챌린지',
    'sm' : 'tab',
    'nso' : 'so:r,p:from20230522to20230720'
    
}
response = requests.get(url,params = param, headers = hrd)
# 출력 코드 사이마다 구분 코드 삽입
print('요청결과')
print(response)             # 요청결과 확인 코드
print('매개 포함 url')
print(response.url)         # 매개인자가 포함된 url형태
print('요청코드')
print(response.status_code) # 요청코드 출력
print('결과 html')
print(response.text)        # 해당 url로의 html코드 출력

요청결과
<Response [200]>
매개 포함 url
https://search.naver.com/search.naver?where=blog&query=%EC%B2%B4%ED%81%AC%EC%9D%B8%EC%B1%8C%EB%A6%B0%EC%A7%80&sm=tab&nso=so%3Ar%2Cp%3Afrom20230522to20230720
요청코드
200
결과 html
<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta property="og:title" content="체크인챌린지 : 네이버 블로그검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="'체크인챌린지'의 네이버 VIEW검색 결과입니다."> <meta name="description" lang="ko" content="'체크인챌린지'의 네이버 VIEW검색 결과입니다."> <title>체크인챌린지 : 네이버 블로그검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><script> if (top.frames.l

- 네이버 검색 페이지에서 블로그로 접근하는 방식이다.
- 데이터 크롤링을 할 때의 문제점을 봐야할 것 같다.

In [49]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

area = soup.find('ul',{'class':'lst_view _list_base'}).find_all('a',{'class':'title_link'})

In [50]:
print(area)

[<a class="title_link" href="https://blog.naver.com/kagama007/223117217017" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=1&amp;i=90000003_0000000000000033F2D28CF9&amp;u='+urlencode(this.href))" target="_blank"><mark>체크인 챌린지</mark> 베트남여행 염소고기 말고 오리고기만 먹을게요~</a>, <a class="title_link" href="https://blog.naver.com/hanlyang_biography/223140596561" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=2&amp;i=90000003_0000000000000033F4374B51&amp;u='+urlencode(this.href))" target="_blank">[블챌][<mark>체크인 챌린지</mark>]이마트 반야월점, 미션! 가자미미역국을... </a>, <a class="title_link" href="https://blog.naver.com/rlagml0403/223151300999" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=3&amp;i=90000003_0000000000000033F4DAA187&amp;u='+urlencode(this.href))" target="_blank">제천 옥순봉 출렁다리 <mark>체크인챌린지</mark></a>, <a class="title_link" href="https://blog.naver.com/joop336/223117203195" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=4&amp;i=90000003_0000000000000033F2D256FB&amp;u='+urlencode(this.href)

In [51]:
for tag in area:
    print(tag)

<a class="title_link" href="https://blog.naver.com/kagama007/223117217017" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=1&amp;i=90000003_0000000000000033F2D28CF9&amp;u='+urlencode(this.href))" target="_blank"><mark>체크인 챌린지</mark> 베트남여행 염소고기 말고 오리고기만 먹을게요~</a>
<a class="title_link" href="https://blog.naver.com/hanlyang_biography/223140596561" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=2&amp;i=90000003_0000000000000033F4374B51&amp;u='+urlencode(this.href))" target="_blank">[블챌][<mark>체크인 챌린지</mark>]이마트 반야월점, 미션! 가자미미역국을... </a>
<a class="title_link" href="https://blog.naver.com/rlagml0403/223151300999" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=3&amp;i=90000003_0000000000000033F4DAA187&amp;u='+urlencode(this.href))" target="_blank">제천 옥순봉 출렁다리 <mark>체크인챌린지</mark></a>
<a class="title_link" href="https://blog.naver.com/joop336/223117203195" onclick="return goOtherCR(this, 'a=blg*a.iblg&amp;r=4&amp;i=90000003_0000000000000033F2D256FB&amp;u='+urlencode(this.href))" t

In [52]:
for tag in area:
    url1 = tag.get('href')
    print("{:}".format(url1))

https://blog.naver.com/kagama007/223117217017
https://blog.naver.com/hanlyang_biography/223140596561
https://blog.naver.com/rlagml0403/223151300999
https://blog.naver.com/joop336/223117203195
https://blog.naver.com/kooltz1018/223156943164
https://blog.naver.com/lovesowzv/223114722803
https://blog.naver.com/asnymse/223160852211
https://blog.naver.com/wonsim105/223109327832
https://blog.naver.com/achellnet/223152673497
https://blog.naver.com/mirinaeverse/223130326453
https://blog.naver.com/ninamath/223112555637
https://blog.naver.com/queen77577/223122908878
https://blog.naver.com/kagama007/223122805613
https://blog.naver.com/dkaldi12/223147185094
https://blog.naver.com/strowwani/223144961274
https://blog.naver.com/rlagml0403/223154365432
https://blog.naver.com/joop336/223160720048
https://blog.naver.com/kooltz1018/223160131594
https://blog.naver.com/asnymse/223155355039
https://blog.naver.com/sun2hwa/223111038006
https://blog.naver.com/webneko/223108736089
https://blog.naver.com/mirinaev

# 네이버 블로그 페이지에서 접근하는 방식

In [21]:
import requests

url = 'https://section.blog.naver.com/Search/Post.naver?pageNo=1'
# url = 'https://search.naver.com/search.naver'
hrd ={'User-Agent': 'Mozilla/5.0', 'referer': 'http://naver.com'}
# 해당 hrd를 requests.get 모듈의 headers 인자로 넘기면 
# 서버에게 외부 프로그램이 아닌 정상적인 형태로 접근하는 것처럼 보일 수 있다고 한다.
param ={
#     'where' : 'blog',
    'rangeType' : 'PERIOD',
    'orderBy' : 'sim',
    'startDate' : '2023-05-22',
    'endDate' : '2023-07-20',
    'keyword' : '체크인챌린지'
}
response = requests.get(url,params = param, headers = hrd)
# 출력 코드 사이마다 구분 코드 삽입
print('요청결과')
print(response)             # 요청결과 확인 코드
print('매개 포함 url')
print(response.url)         # 매개인자가 포함된 url형태
print('요청코드')
print(response.status_code) # 요청코드 출력
# print('결과 html')
# print(response.text)        # 해당 url로의 html코드 출력

요청결과
<Response [200]>
매개 포함 url
https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2023-05-22&endDate=2023-07-20&keyword=%EC%B2%B4%ED%81%AC%EC%9D%B8%EC%B1%8C%EB%A6%B0%EC%A7%80
요청코드
200


- page no1 말고 다른 url을 삽입했을 때의 방법을 찾아봐야할 것 같다.